# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
!pip install pymongo

In [23]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [24]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [25]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'local', 'uk_food']

In [26]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [27]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [28]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('65b7813835a1e3f4a38064cf'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Halal Restaurant',
 'BusinessTypeID': 1,
 'geocode': {'longitude': None, 'latitude': None},
 'RatingValue': None}

In [29]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [30]:
# Create a dictionary for the new restaurant data
new_restaurant = {'BusinessName': 'Penang Flavours', 'BusinessType': 'Halal Restaurant'}

In [31]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('65b78da149c6a5ad838cfb7e'), acknowledged=True)

In [32]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName': 'Penang Flavours'})

{'_id': ObjectId('65b7813835a1e3f4a38064cf'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Halal Restaurant',
 'BusinessTypeID': 1,
 'geocode': {'longitude': None, 'latitude': None},
 'RatingValue': None}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [33]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
establishments.find_one({'BusinessType': 'Restaurant/Cafe/Canteen'}, {'BusinessTypeID': 1, 'BusinessType': 1})


3. Update the new restaurant with the `BusinessTypeID` you found.

In [34]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({'BusinessName': 'Penang Flavours'},
                          {'$set': {'BusinessTypeID': 1}
                          })

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [35]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName': 'Penang Flavours'})

{'_id': ObjectId('65b7813835a1e3f4a38064cf'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Halal Restaurant',
 'BusinessTypeID': 1,
 'geocode': {'longitude': None, 'latitude': None},
 'RatingValue': None}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [36]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': 'Dover'})

0

In [37]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [38]:
# Check if any remaining documents include Dover
establishments.count_documents({'LocalAuthorityName': 'Dover'})

0

In [39]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('65b7813835a1e3f4a38064cf'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Halal Restaurant',
 'BusinessTypeID': 1,
 'geocode': {'longitude': None, 'latitude': None},
 'RatingValue': None}

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [40]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"}}} ])
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDecimal': "$geocode.latitude"}}} ])

UpdateResult({'n': 3, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [41]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [42]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

UpdateResult({'n': 3, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [43]:
# Check that the coordinates and rating value are now numbers
# Initialize flags to check the data types
rating_values_are_ints = True
latitudes_are_decimals = True
longitudes_are_decimals = True

# Iterate over each document in the establishments collection
for document in establishments.find():
    # Skip documents where 'RatingValue' is 'none'
    if 'RatingValue' in document and document['RatingValue'] == 'none':
        continue

    # Check if 'RatingValue' is an integer
    if 'RatingValue' in document and not isinstance(document['RatingValue'], int):
        rating_values_are_ints = False

    # Check if 'latitude' and 'longitude' exist and are decimals
    if 'latitude' in document and 'longitude' in document:
        if not (isinstance(document['latitude'], Decimal) and isinstance(document['longitude'], Decimal)):
            latitudes_are_decimals = False
            longitudes_are_decimals = False

# Print the results
if rating_values_are_ints:
    print("All RatingValues are integers.")
else:
    print("All RatingValues that do not have 'none' as its rating are integers.")

if latitudes_are_decimals and longitudes_are_decimals:
    print("All latitudes and longitudes are decimals.")
else:
    print("Not all latitudes and longitudes are decimals.")

All RatingValues that do not have 'none' as its rating are integers.
All latitudes and longitudes are decimals.
